In [1]:
%load_ext sql
%sql sqlite:///chinook.db

'Connected: @chinook.db'

## Задание 1
Вывести среднюю продолжительность плейлиста в секундах *(продолжительность плейлиста - сумма длин треков в нём)*.

In [2]:
%%sql
--
SELECT
    avg(Time)
FROM
    (
    SELECT
        sum(Milliseconds) / 1000 AS Time
    FROM
        tracks LEFT JOIN (SELECT TrackId, PlaylistId FROM playlist_track) USING (TrackId)
    GROUP BY
        PlaylistId
    )


 * sqlite:///chinook.db
Done.


avg(Time)
230150.2142857143


## Задание 2
Вывести альбом, который является минимальным по продолжительности среди альбомов со стоимостью, большей средней цены за альбом *(аналогично цена альбома - сумма цен треков)*.

In [4]:
%%sql
--
SELECT
    Title, min(Time)
FROM 
    (
    SELECT
        s.Title, s.Cost, s.Time
    FROM
        (
        SELECT
            Title, sum(UnitPrice) AS Cost, sum(Milliseconds) / 1000 AS Time
        FROM
            tracks LEFT JOIN (SELECT * FROM albums) USING (AlbumId)
        GROUP BY
            Title
        ) s,
        (
        SELECT 
            avg(Cost) AS avg_cost
        FROM
            (
            SELECT
                Title, sum(UnitPrice) AS Cost
            FROM
                tracks LEFT JOIN (SELECT * FROM albums) USING (AlbumId)
            GROUP BY
                Title
            )
        )
    WHERE
        s.Cost > avg_cost
    )


 * sqlite:///chinook.db
Done.


Title,min(Time)
BackBeat Soundtrack,1615


## Задание 3
Вывести общую стоимость треков, приобретённых компаниями, которые обслуживались сотрудниками, нанятыми после 3 сентября 2002 года.

In [5]:
%%sql
--
SELECT
    sum(UnitPrice)
FROM
    tracks
        LEFT JOIN (SELECT TrackId,    InvoiceId                FROM invoice_items) USING (TrackId)
        LEFT JOIN (SELECT InvoiceId,  CustomerId               FROM invoices)      USING (InvoiceId)
        LEFT JOIN (SELECT CustomerId, SupportRepId,   Company  FROM customers)     USING (CustomerId)
        LEFT JOIN (SELECT EmployeeId AS SupportRepId, HireDate FROM employees)     USING (SupportRepId)   
WHERE 
    (Company IS NOT null) AND (HireDate > "2002-09-3")

 * sqlite:///chinook.db
Done.


sum(UnitPrice)
230.72000000000062


## Задание 4
Для каждого альбома определить все жанры песен, которые в нём есть и количество песен по каждому жанру (формат вывода: альбом, жанр, количество).
Выводить только те альбомы, где жанров больше одного.

In [17]:
%%sql
--
SELECT
    AlbumId, Title, Genre, count(TrackId) AS Count
FROM
    (SELECT
        *
    FROM
        tracks
            LEFT JOIN (SELECT GenreId, Name AS Genre FROM genres) USING (GenreId)
            LEFT JOIN (SELECT AlbumId, Title         FROM albums) USING (AlbumId)
    ) t,
    (
    SELECT 
        AlbumId AS Ids
    FROM
        tracks
            LEFT JOIN (SELECT GenreId, Name AS Genre FROM genres) USING (GenreId)
            LEFT JOIN (SELECT AlbumId, Title         FROM albums) USING (AlbumId)
    GROUP BY
        AlbumId
    HAVING
        count(distinct Genre) > 1
    )
WHERE
    t.AlbumId = Ids
GROUP BY
    t.AlbumId, t.Genre
LIMIT 
    10


 * sqlite:///chinook.db
Done.


AlbumId,Title,Genre,Count
73,Unplugged,Blues,14
73,Unplugged,Latin,16
102,Live After Death,Heavy Metal,7
102,Live After Death,Metal,11
109,Rock In Rio [CD2],Metal,1
109,Rock In Rio [CD2],Rock,8
112,The Number of The Beast,Metal,7
112,The Number of The Beast,Rock,1
141,Greatest Hits,Metal,14
141,Greatest Hits,Reggae,13


In [7]:
%%sql
--
SELECT 
    AlbumId
FROM
    tracks LEFT JOIN (SELECT GenreId, Name AS Genre FROM genres) USING (GenreId)
           LEFT JOIN (SELECT AlbumId, Title         FROM albums) USING (AlbumId)
GROUP BY
    AlbumId
HAVING 
    count(distinct Genre) > 1

 * sqlite:///chinook.db
Done.


AlbumId
73
102
109
112
141
227
228
229
231
251


## Задание 5
Для каждого артиста (таблицы Artists), у которых есть песни нескольких жанров, найти жанр, по которому он заработал наибольшее количество денег, а также вывести соотношение суммы, заработанной артистом по этому жанру к общей сумме заработанных денег. Если по нескольким жанрам заработано одинаковое количество денег, максимальное для артиста, то выбирается жанр раньше по алфавиту. Формат вывода: артист, жанр, соотношение.

In [31]:
%%sql
SELECT 
    Artist, Genre, max(Sum) AS Max, max(Sum) / sum(Sum) AS Ratio
FROM
    (
    SELECT
        Artist, Genre, sum(UnitPrice) AS Sum
    FROM
        (
        SELECT
            ArtistId
        FROM
            tracks LEFT JOIN (SELECT GenreId, Name AS Genre   FROM genres )  USING (GenreId) 
                   LEFT JOIN (SELECT AlbumId, Title, ArtistId FROM albums)   USING (AlbumId)
                   LEFT JOIN (SELECT ArtistId, Name AS Artist FROM artists)  USING (ArtistId)
        GROUP BY 
            ArtistId
        HAVING 
            count(distinct Genre) > 1
        ) ids,
        (
        SELECT
            *
       FROM
           tracks
               LEFT JOIN (SELECT GenreId, Name AS Genre   FROM genres)  USING (GenreId)
               LEFT JOIN (SELECT AlbumId, Title, ArtistId FROM albums)  USING (AlbumId)
               LEFT JOIN (SELECT ArtistId, Name AS Artist FROM artists) USING (ArtistId)
        ) t
    WHERE
        t.ArtistId = ids.ArtistId
    GROUP BY 
        t.ArtistId, t.Genre
    )
GROUP BY
    Artist
ORDER BY 
    Genre  ASC,
    Artist ASC 
LIMIT 10

 * sqlite:///chinook.db
Done.


Artist,Genre,Max,Ratio
Audioslave,Alternative,13.860000000000001,0.35000000000000003
Faith No More,Alternative & Punk,36.62999999999999,0.7115384615384615
R.E.M.,Alternative & Punk,26.729999999999986,0.6585365853658535
Eric Clapton,Blues,31.67999999999998,0.6666666666666665
Heroes,Drama,39.8,0.8695652173913043
Jamiroquai,Electronica/Dance,10.89,0.34375
Antônio Carlos Jobim,Latin,16.830000000000002,0.5483870967741935
Gilberto Gil,Latin,14.850000000000001,0.46875000000000006
Various Artists,Latin,27.719999999999985,0.4999999999999999
Iron Maiden,Metal,94.04999999999988,0.4460093896713613


In [13]:
%%sql
--
SELECT
    Artist, Genre, count(distinct TrackId), sum(UnitPrice) AS Sum
FROM
    (
    SELECT
        ArtistId
    FROM
        tracks
            LEFT JOIN (SELECT GenreId, Name AS Genre   FROM genres)  USING (GenreId)
            LEFT JOIN (SELECT AlbumId, Title, ArtistId FROM albums)  USING (AlbumId)
            LEFT JOIN (SELECT ArtistId, Name AS Artist FROM artists) USING (ArtistId)
    GROUP BY
        ArtistId
    HAVING 
        count(distinct Genre) > 1) ids,
    (
    SELECT
        *
    FROM
        tracks
            LEFT JOIN (SELECT GenreId, Name AS Genre   FROM genres)  USING (GenreId)
            LEFT JOIN (SELECT AlbumId, Title, ArtistId FROM albums)  USING (AlbumId)
            LEFT JOIN (SELECT ArtistId, Name AS Artist FROM artists) USING (ArtistId)
    ) t
WHERE
    t.ArtistId = ids.ArtistId
GROUP BY
    t.ArtistId, t.Genre
LIMIT 
    10

 * sqlite:///chinook.db
Done.


Artist,Genre,count(distinct TrackId),Sum
Antônio Carlos Jobim,Jazz,14,13.860000000000001
Antônio Carlos Jobim,Latin,17,16.830000000000002
Audioslave,Alternative,14,13.860000000000001
Audioslave,Alternative & Punk,12,11.88
Audioslave,Rock,14,13.860000000000001
Various Artists,Latin,28,27.719999999999985
Various Artists,Pop,14,13.860000000000001
Various Artists,Soundtrack,14,13.860000000000001
Gilberto Gil,Jazz,3,2.9699999999999998
Gilberto Gil,Latin,15,14.850000000000001
